In [291]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import itertools

from simulate import init, simulate_n_times

In [292]:
em, l1 = init()
print(simulate_n_times(n_sim=100, em=em, l1=l1, la=0.2, n=8))

(0.3486012255792793, 1.756556494794709, 0.1758941230466079)


In [293]:
mask = pd.DataFrame(columns=['x0', 'lambda_customers', 't_k1', 'mu_k2', 't_k3', 'n_k2', 'lc*tk1', 'lc*mu2'],
                    index=range(8))
mask.iloc[:, 0] = 1
mask.iloc[:, 1] = np.tile([1, -1], 4)
mask.iloc[:, 2] = np.tile([1, 1, -1, -1], 2)
mask.iloc[:, 3] = [1, 1, 1, 1, -1, -1, -1, -1]
mask.iloc[:, 4] = mask.iloc[:, 1] * mask.iloc[:, 2] * mask.iloc[:, 3]
mask.iloc[:, 5] = mask.iloc[:, 2] * mask.iloc[:, 3]

In [294]:
# Comment to have a partial experiment

mask = pd.DataFrame(columns=['lambda_customers', 't_k1', 'mu_k2', 't_k3', 'n_k2'],
                    index=range(32),
                    data=itertools.product([-1, 1], repeat=5))

mask['x0'] = 1
mask['lc*tk1'] = None
mask['lc*mu2'] = None
mask = mask.reindex(columns=['x0', 'lambda_customers', 't_k1', 'mu_k2', 't_k3', 'n_k2', 'lc*tk1', 'lc*mu2'])
print(mask)

    x0  lambda_customers  t_k1  mu_k2  t_k3  n_k2 lc*tk1 lc*mu2
0    1                -1    -1     -1    -1    -1   None   None
1    1                -1    -1     -1    -1     1   None   None
2    1                -1    -1     -1     1    -1   None   None
3    1                -1    -1     -1     1     1   None   None
4    1                -1    -1      1    -1    -1   None   None
5    1                -1    -1      1    -1     1   None   None
6    1                -1    -1      1     1    -1   None   None
7    1                -1    -1      1     1     1   None   None
8    1                -1     1     -1    -1    -1   None   None
9    1                -1     1     -1    -1     1   None   None
10   1                -1     1     -1     1    -1   None   None
11   1                -1     1     -1     1     1   None   None
12   1                -1     1      1    -1    -1   None   None
13   1                -1     1      1    -1     1   None   None
14   1                -1     1      1   

In [295]:
mask.iloc[:, 6] = mask.iloc[:, 1] * mask.iloc[:, 2]
mask.iloc[:, 7] = mask.iloc[:, 1] * mask.iloc[:, 3]
print(mask)

    x0  lambda_customers  t_k1  mu_k2  t_k3  n_k2  lc*tk1  lc*mu2
0    1                -1    -1     -1    -1    -1       1       1
1    1                -1    -1     -1    -1     1       1       1
2    1                -1    -1     -1     1    -1       1       1
3    1                -1    -1     -1     1     1       1       1
4    1                -1    -1      1    -1    -1       1      -1
5    1                -1    -1      1    -1     1       1      -1
6    1                -1    -1      1     1    -1       1      -1
7    1                -1    -1      1     1     1       1      -1
8    1                -1     1     -1    -1    -1      -1       1
9    1                -1     1     -1    -1     1      -1       1
10   1                -1     1     -1     1    -1      -1       1
11   1                -1     1     -1     1     1      -1       1
12   1                -1     1      1    -1    -1      -1      -1
13   1                -1     1      1    -1     1      -1      -1
14   1    

In [296]:
translator = pd.DataFrame(columns=['lambda_customers', 't_k1', 'mu_k2', 't_k3', 'n_k2'],
                          index=['min', 'max'],
                          data=[[0.1, 1, 1 / 8, 1, 10],
                                [0.4, 3, 1 / 2, 3, 20]])

translator.loc[:, 'lc*tk1'] = translator.loc[:, 'lambda_customers'] * translator.loc[:, 't_k1']
translator.loc[:, 'lc*mu2'] = translator.loc[:, 'lambda_customers'] * translator.loc[:, 'mu_k2']

translator.loc['half_diff', :] = (translator.loc['max', :] - translator.loc['min', :]) / 2
translator.loc['avg', :] = (translator.loc['max', :] + translator.loc['min', :]) / 2

translator

,lambda_customers,t_k1,mu_k2,t_k3,n_k2,lc*tk1,lc*mu2
min,0.10,1.0,0.1250,1.0,10.0,0.10,0.01250
max,0.40,3.0,0.5000,3.0,20.0,1.20,0.20000
half_diff,0.15,1.0,0.1875,1.0,5.0,0.55,0.09375
avg,0.25,2.0,0.3125,2.0,15.0,0.65,0.10625


In [297]:
unmasked_values = translator.loc['half_diff', :] * mask + translator.loc['avg', :]
unmasked_values['x0'] = 1
unmasked_values = unmasked_values.reindex(
    columns=['x0', 'lambda_customers', 't_k1', 'mu_k2', 't_k3', 'n_k2', 'lc*tk1', 'lc*mu2'])
unmasked_values

,x0,lambda_customers,t_k1,mu_k2,t_k3,n_k2,lc*tk1,lc*mu2
0,1,0.1,1.0,0.125,1.0,10.0,1.2,0.2000
1,1,0.1,1.0,0.125,1.0,20.0,1.2,0.2000
2,1,0.1,1.0,0.125,3.0,10.0,1.2,0.2000
3,1,0.1,1.0,0.125,3.0,20.0,1.2,0.2000
4,1,0.1,1.0,0.500,1.0,10.0,1.2,0.0125
5,1,0.1,1.0,0.500,1.0,20.0,1.2,0.0125
6,1,0.1,1.0,0.500,3.0,10.0,1.2,0.0125
7,1,0.1,1.0,0.500,3.0,20.0,1.2,0.0125
8,1,0.1,3.0,0.125,1.0,10.0,0.1,0.2000
9,1,0.1,3.0,0.125,1.0,20.0,0.1,0.2000


In [298]:
results = []
for index, row in unmasked_values.iterrows():
    lambda_customers = row['lambda_customers']
    t_k1 = row['t_k1']
    mu_k2 = row['mu_k2']
    t_k3 = row['t_k3']
    n_k2 = row['n_k2']

    em, l1 = init(t_k1=t_k1, mu_k2=mu_k2, t_k3=t_k3, capacity_q2=n_k2)
    result = simulate_n_times(n_sim=100, em=em, l1=l1, la=lambda_customers, n=500)

    results.append(result)

# results_df = pd.DataFrame(results, columns=['rejection_rate', 'avg_wait_time', 'avg_q_len'])
results_df = pd.Series([r[1] for r in results], name='avg_wait_time')
df = pd.concat([unmasked_values, results_df], axis=1)
df

,x0,lambda_customers,t_k1,mu_k2,t_k3,n_k2,lc*tk1,lc*mu2,avg_wait_time
0,1,0.1,1.0,0.125,1.0,10.0,1.2,0.2000,16.289378
1,1,0.1,1.0,0.125,1.0,20.0,1.2,0.2000,20.570214
2,1,0.1,1.0,0.125,3.0,10.0,1.2,0.2000,19.046584
3,1,0.1,1.0,0.125,3.0,20.0,1.2,0.2000,19.510535
4,1,0.1,1.0,0.500,1.0,10.0,1.2,0.0125,0.264648
5,1,0.1,1.0,0.500,1.0,20.0,1.2,0.0125,0.266130
6,1,0.1,1.0,0.500,3.0,10.0,1.2,0.0125,0.345242
7,1,0.1,1.0,0.500,3.0,20.0,1.2,0.0125,0.349545
8,1,0.1,3.0,0.125,1.0,10.0,0.1,0.2000,8.147948
9,1,0.1,3.0,0.125,1.0,20.0,0.1,0.2000,8.692225


In [299]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [300]:
print(sm.OLS(y, X).fit().summary())

                            OLS Regression Results                            
Dep. Variable:          avg_wait_time   R-squared:                       0.904
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     32.22
Date:                Tue, 02 Jan 2024   Prob (F-statistic):           1.04e-10
Time:                        21:30:23   Log-Likelihood:                -129.96
No. Observations:                  32   AIC:                             275.9
Df Residuals:                      24   BIC:                             287.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
x0                  42.0513     14.684  